# Are there concerns about what instructors are using?
* Are there unlisenced products instructors are using?
* Who is using New Box View? 
* Force completion final exams? 

In [2]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
bbuser, bbpass, bbhost = ge('bbuser'), ge('bbpass'), ge('bbhost')
con_string = f'oracle+cx_oracle://{bbuser}:{bbpass}@{bbhost}'
engine = create_engine(con_string, max_identifier_length=128)

## Which courses are using YouTube?

In [6]:
query = """
SELECT SUBSTR(cm.COURSE_ID,0,4) term, count(cm.COURSE_ID) num_courses
FROM BB_BB60.COURSE_MAIN cm
JOIN (select crsmain_pk1, count(pk1) as youtube
           from bb_bb60.course_contents
           where cnthndlr_handle = 'resource/x-bb-youtube-mashup'
           group by crsmain_pk1) youtube on youtube.crsmain_pk1 = cm.pk1
WHERE cm.COURSE_ID LIKE '42%'
GROUP BY SUBSTR(cm.COURSE_ID,0,4)
ORDER BY term
"""
pd.read_sql(query, con=engine).head()

,term,num_courses
0,4202,301
1,4206,95
2,4209,21


In [13]:
query = """
SELECT cm.COURSE_ID, count(youtube) youtube_count
FROM BB_BB60.COURSE_MAIN cm
JOIN (select crsmain_pk1, count(pk1) as youtube
           from bb_bb60.course_contents
           where cnthndlr_handle = 'resource/x-bb-youtube-mashup'
           group by crsmain_pk1) youtube on youtube.crsmain_pk1 = cm.pk1
WHERE cm.COURSE_ID LIKE '4206-%'
GROUP BY cm.COURSE_ID
ORDER BY youtube_count desc
"""
pd.read_sql(query, con=engine).head()

,course_id,youtube_count
0,4206-74310,1
1,4206-77659,1
2,4206-73407,1
3,4206-75321,1
4,4206-75609,1


## Which courses are using monitor?
query = """
SELECT cm.COURSE_ID, 
"""
pd.read_sql(query, con=engine).head()